In [1]:
from datetime import datetime, timedelta
import netCDF4
import numpy as np
import bisect
import math

# 1) Current implementation of subsetting from nc file example

In [2]:
def get_current_data_subset(nc_file, x_0, x_T, deg_around_x0_xT_box, fixed_time=None,
                            temporal_stride=1, temp_horizon_in_h=None):
    """ Function to read a subset of the nc_file current data bounded by a box spanned by the x_0 and x_T points.
    Inputs:
        nc_file                 full path to nc file
        x_0                     [lon, lat, charge, timestamp in POSIX]
        x_T                     [lon, lat] goal locations
        deg_around_x0_xT_box    float, buffer around the box in degrees
        fixed_time              if None returns time-varying currents, 
                                otherwise datetime object of the fixed time -> returns ocean current grid at or before time
                                the time of x_0 is then ignored
        temporal_stride         int, if a stride of the temporal values is used (every temporal_stride hours)
        temp_horizon            if None: all available time of the file will be provided
                                otherwise float, maximum temp_horizon to look ahead of x_0 time in hours
                                
    Outputs:
        grids_dict              dict containing x_grid, y_grid, t_grid, fixed_time_idx
        u_data                  [T, Y, X] matrix of the ocean currents in x direction in m/s
        v_data                  [T, Y, X] matrix of the ocean currents in y direction in m/s
        
    """
    
    f = netCDF4.Dataset(nc_file)

    # extract positiond & start_time for the indexing
    x_0_pos = x_0[:2]
    x_0_posix_time = x_0[3]
    x_T = x_T[:2]

    # Step 1: get the grids
    xgrid = f.variables['lon'][:]
    ygrid = f.variables['lat'][:]
    t_grid = f.variables['time'][:] # not this is in hours from HYCOM data!
    
    # this is needed because the time origin in hindcast and forecase nc files is different. Very handcrafted.
    try:
        time_origin = datetime.strptime(f.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
    except:
        time_origin = datetime.strptime(f.variables['time'].__dict__['units'] + ' +0000',
                                                 'hours since %Y-%m-%d %H:%M:%S.000 UTC %z')

    # Step 2: find the sub-setting
    # find the lat & lon sub-set bounds
    lon_bnds = [min(x_0_pos[0], x_T[0]) - deg_around_x0_xT_box, max(x_0_pos[0], x_T[0]) + deg_around_x0_xT_box]
    lat_bnds = [min(x_0_pos[1], x_T[1]) - deg_around_x0_xT_box, max(x_0_pos[1], x_T[1]) + deg_around_x0_xT_box]

    # get the respective indices from the grids
    ygrid_inds = np.where((ygrid >= lat_bnds[0]) & (ygrid <= lat_bnds[1]))[0]
    xgrid_inds = np.where((xgrid >= lon_bnds[0]) & (xgrid <= lon_bnds[1]))[0]

    # for time indexing transform to POSIX time
    abs_t_grid = [(time_origin + timedelta(hours=X)).timestamp() for X in t_grid.data]
    
    # get the idx of the value left of the demanded time (for interpolation function)
    t_start_idx = bisect.bisect_right(abs_t_grid, x_0_posix_time) - 1
    if t_start_idx == len(abs_t_grid) - 1 or t_start_idx == -1:
        raise ValueError("Requested subset time is outside of the nc4 file.")

    # get the max time if provided as input
    if temp_horizon_in_h is None:   # all data provided
        t_end_idx = len(abs_t_grid)-1
    else:
        t_end_idx = bisect.bisect_right(abs_t_grid, x_0_posix_time + temp_horizon_in_h*3600.)
        if t_end_idx == len(abs_t_grid):
            raise ValueError("nc4 file does not contain requested temporal horizon.")

    # fixed time logic if necessary
    if fixed_time is None:
        slice_for_time_dim = np.s_[t_start_idx:(t_end_idx+1):temporal_stride]
        fixed_time_idx = None
    else:
        fixed_time_idx = bisect.bisect_right(abs_t_grid, fixed_time.timestamp()) - 1
        slice_for_time_dim = np.s_[fixed_time_idx]

    # Step 2: extract data
    # raw water_u is [tdim, zdim, ydim, xdim]
    if len(f.variables['water_u'].shape) == 4:  # if there is a depth dimension in the dataset
        u_data = f.variables['water_u'][slice_for_time_dim, 0, ygrid_inds, xgrid_inds]
        v_data = f.variables['water_v'][slice_for_time_dim, 0, ygrid_inds, xgrid_inds]
    # raw water_u is [tdim, ydim, xdim]
    elif len(f.variables['water_u'].shape) == 3:  # if there is no depth dimension in the dataset
        u_data = f.variables['water_u'][slice_for_time_dim, ygrid_inds, xgrid_inds]
        v_data = f.variables['water_v'][slice_for_time_dim, ygrid_inds, xgrid_inds]
    else:
        raise ValueError("Current data in nc file has neither 3 nor 4 dimensions. Check file.")

    # create dict to output
    grids_dict = {'x_grid': xgrid[xgrid_inds], 'y_grid': ygrid[ygrid_inds],
                  't_grid': abs_t_grid[slice_for_time_dim], 'fixed_time_idx': fixed_time_idx}

    # log what data has been subsetted
    if fixed_time is None:
        print("Subsetted data from {start} to {end} in {n_steps} time steps of {time:.2f} hour(s) resolution".format(
            start=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            end=datetime.utcfromtimestamp(grids_dict['t_grid'][-1]).strftime('%Y-%m-%d %H:%M:%S UTC'),
            n_steps=len(grids_dict['t_grid']), time=(grids_dict['t_grid'][1] - grids_dict['t_grid'][0])/3600.))
    else:
        print("Subsetted data to fixed time at: {time}".format(
            time=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC')))

    #TODO: we replace the masked array with fill value 0 because otherwise interpolation doesn't work.
    # Though that means we cannot anymore detect if we're on land or not (need a way to do that/detect stranding)
    # not sure yet if we'll do it in the simulator or where.
    return grids_dict, u_data.filled(fill_value=0.), v_data.filled(fill_value=0.)

# 2) C3 function loading data using file-based approach

In [3]:
def get_current_data_subset_from_c3(
    t_interval, #temp_res_in_h,   ----> separate function
    lat_interval, #lat_res_in_deg,
    lon_interval, #lon_res_in_deg,
    #depth_interval_to_avg_over
):
    
    # scipy.interpolate.interp1d
    
    """ Function to get a subset of current data via the C3 data integration.
    
    Inputs:
        t_interval              if time-varying: [t_0, t_T] in POSIX time
                                where t_0 and t_T are the start and end timestamps respectively
                                if fixed_time:   [fixed_timestamp] in POSIX
        temp_res_in_h           which temporal resolution the time-axis should have
                                e.g. if temp_res_in_h = 1, t_grid = [t_0, t_0 + 3600s, ... t_T]
                                if temp_res_in_h = 5,      t_grid = [t_0, t_0 + 5*3600s, ... t_T]
                                if temp_res_in_h = 0.5,      t_grid = [t_0, t_0 + 1800s, ... t_T]
                                => so either averaging or interpolation needs to be done in the backend
        lat_interval            [y_lower, y_upper] in degrees
        lat_res_in_deg          which spatial resolution in y direction in degrees
                                e.g. if lat_res_in_deg = 1, y_grid = [y_lower, y_lower + 1, ... y_upper]
                                 => so either averaging or interpolation needs to be done in the backend
        lon_interval            [x_lower, x_upper] in degrees
        lon_res_in_deg          which spatial resolution in x direction in degrees
                                e.g. if lon_res_in_deg = 1, x_grid = [x_lower, x_lower + 1, ... x_upper]
                                 => so either averaging or interpolation needs to be done in the backend
        depth_interval_to_avg_over
                                Interval to average over the current dimension in meters
                                e.g. [0, 10] then the currents are averaged over the depth 0-10m.
                                
    Outputs:
        grids_dict              dict containing x_grid, y_grid, t_grid
        u_data                  [T, Y, X] matrix of the ocean currents in x direction in m/s
        v_data                  [T, Y, X] matrix of the ocean currents in y direction in m/s
    """
    
    # Step 1: get required file references and data from C3 file DB
    # Step 1.1: Getting time and formatting for the db query
    start_date = datetime.utcfromtimestamp(t_interval[0]).strftime("%Y-%m-%d")
    end_date = datetime.utcfromtimestamp(t_interval[1]).strftime("%Y-%m-%d")

    
    # Step 1.2: Getting correct range of nc files from database
    filter_string = 'start>=' + '"'+ start_date + '"' + ' && end<=' + '"' + end_date + "T23:00:00.000" + '"'
    objs_list = c3.HindcastFile.fetch({'filter':filter_string, "order": "start"}).objs
    if objs_list is None:
        raise ValueError("No files in the database for the selected t_interval")
    
    # Step 1.3: extract url and start list from the query results
    urls_list = [obj.file.url for obj in objs_list]
    start_list = [obj.start for obj in objs_list]
    
    # Step 2: Prepare the stacking loop by getting the x, y grids and subsetting indices in x, y 
    # Note: these stay constant across files in this case where all files have same lat-lon range
    
    # Step 2.1: open the file and get the x and y grid
    f = c3.HycomUtil.nc_open(urls_list[0])
    xgrid = f.variables['lon'][:].data
    ygrid = f.variables['lat'][:].data
    
    # Step 2.2: get the respective indices of the lat, lon subset from the file grids
    ygrid_inds = np.where((ygrid >= lat_interval[0]) & (ygrid <= lat_interval[1]))[0]
    xgrid_inds = np.where((xgrid >= lon_interval[0]) & (xgrid <= lon_interval[1]))[0]
    
    # Step 2.3 initialze t_grid stacking variable
    full_t_grid = []

    # Step 3: iterate over all files in order and stack the current data and absolute t_grids
    for idx in range(len(start_list)):
        # Step 3.0: load the current data file
        f = c3.HycomUtil.nc_open(urls_list[idx])
        # set the default start and end time
        start_hr, end_hr = 0, 24
        
        # Step 3.1: do the time-subsetting
        #Case 1: file is first -- get data from the file from the hour before or at t_0
        if idx == 0:
            start_hr = math.floor((t_interval[0] - start_list[idx].timestamp())/3600)
        #Case 2: file is last -- get data from file until or after the hour t_T
        if idx == len(start_list)-1:
            end_hr = math.ceil((t_interval[1] - start_list[idx].timestamp())/3600)+1

        # Step 3.2: extract data from the file
        u_data = f.variables['water_u'][start_hr:end_hr, 0, ygrid_inds, xgrid_inds]
        v_data = f.variables['water_v'][start_hr:end_hr, 0, ygrid_inds, xgrid_inds]

        # Step 3.3: stack the sub-setted abs_t_grid and current data
        full_t_grid = full_t_grid + [start_list[idx].timestamp() + i*3600 for i in range(start_hr, end_hr)]
        
        if idx == 0:
            full_u_data = u_data
            full_v_data = v_data
        else:
            full_u_data = np.concatenate((full_u_data, u_data), axis=0)
            full_v_data = np.concatenate((full_v_data, v_data), axis=0)

    # Step 4: create dict to output
    grids_dict = {'x_grid': xgrid[xgrid_inds], 'y_grid': ygrid[ygrid_inds], 't_grid': full_t_grid}
    
    # Step 5: # log what data has been subsetted
    print("Subsetted data from {start} to {end} in {n_steps} time steps of {time:.2f} hour(s) resolution".format(
        start=datetime.utcfromtimestamp(grids_dict['t_grid'][0]).strftime('%Y-%m-%d %H:%M:%S UTC'),
        end=datetime.utcfromtimestamp(grids_dict['t_grid'][-1]).strftime('%Y-%m-%d %H:%M:%S UTC'),
        n_steps=len(grids_dict['t_grid']), time=(grids_dict['t_grid'][1] - grids_dict['t_grid'][0])/3600.))

    # Step 6: return the grids_dict and the stacked data
    # TODO: currently, we just do fill_value =0 but then we can't detect if we're on land. 
    # We need a way to do that in the simulator, doing it via the currents could be one way.
    return grids_dict, full_u_data.filled(fill_value=0.), full_v_data.filled(fill_value=0.)

# 3) Sanity check if data directly from subsetted file and via c3 files is same

In [4]:
# settings for directly from subsetted file and old approach
hindcast_file = 'Sanity_check_data.nc4'
x_0 = [-90.0, 24.0, 1, 1630584000.0]  # lon, lat, battery, posix_time
x_T = [-89.0, 25.0]
deg_around_x0_xT_box = 0.
fixed_time = None
temporal_stride = 1

# function callfile_
file_grids_dict, file_u_data, file_v_data = get_current_data_subset(hindcast_file,
                                                     x_0, x_T,
                                                     deg_around_x0_xT_box,
                                                     fixed_time,
                                                     temporal_stride,
                                                              temp_horizon_in_h=None)

Subsetted data from 2021-09-02 12:00:00 UTC to 2021-09-03 12:00:00 UTC in 25 time steps of 1.00 hour(s) resolution


In [5]:
# running same query via the data pipeline using the files from the C3 database
lon_interval = [-90,-89]
lat_interval = [24, 25]
# t_interval = [1630584000.0, 1630670400.0] # that's two days 2021-09-02-12 to 2021-09-04-12
t_interval = [file_grids_dict['t_grid'][0], file_grids_dict['t_grid'][-1]] # that's two days 2021-09-02-12 to 2021-09-04-12
c3_grids_dict, c3_u_data, c3_v_data = get_current_data_subset_from_c3(t_interval, lat_interval, lon_interval)

Subsetted data from 2021-09-02 12:00:00 UTC to 2021-09-03 12:00:00 UTC in 25 time steps of 1.00 hour(s) resolution


In [6]:
# compare if they output matrices are the same (they should)
np.all(file_u_data==c3_u_data)

True